In [7]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pd
import tensorflow as tf

Matplotlib created a temporary cache directory at C:\Users\Acer\AppData\Local\Temp\matplotlib-hc8yibp1 because the default path (C:\Users\Acer\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [8]:
df = pd.read_csv('fer2013.csv')
print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [9]:
X_train = []
y_train = []
X_test = []
y_test = []
for index, row in df.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

In [10]:
X_train[0]

array(['70', '80', '82', ..., '106', '109', '82'], dtype='<U3')

In [11]:
X_train = np.array(X_train, dtype = 'uint8')
y_train = np.array(y_train, dtype = 'uint8')
X_test = np.array(X_test, dtype = 'uint8')
y_test = np.array(y_test, dtype = 'uint8')

In [12]:
import keras
from keras.utils import to_categorical
y_train= to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [13]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [14]:
from keras.preprocessing.image import ImageDataGenerator 
datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode = 'nearest')

In [15]:
testgen = ImageDataGenerator(rescale=1./255)
datagen.fit(X_train)
batch_size = 64

In [16]:
train_flow = datagen.flow(X_train, y_train, batch_size=batch_size) 
test_flow = testgen.flow(X_test, y_test, batch_size=batch_size)

In [17]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

In [18]:
def FER_Model(input_shape=(48,48,1)):
    # first input model
    visible = Input(shape=input_shape, name='input')
    num_classes = 7
    #the 1-st block
    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)#the 2-nd block
    conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
    conv2_2 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)#the 3-rd block
    conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
    conv3_1 = BatchNormalization()(conv3_1)
    conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
    conv3_2 = BatchNormalization()(conv3_2)
    conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
    conv3_3 = BatchNormalization()(conv3_3)
    conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
    conv3_4 = BatchNormalization()(conv3_4)
    pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
    drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)#the 4-th block
    conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
    conv4_1 = BatchNormalization()(conv4_1)
    conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
    conv4_2 = BatchNormalization()(conv4_2)
    conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
    conv4_3 = BatchNormalization()(conv4_3)
    conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
    conv4_4 = BatchNormalization()(conv4_4)
    pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
    drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)
    
    #the 5-th block
    conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
    conv5_1 = BatchNormalization()(conv5_1)
    conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
    conv5_2 = BatchNormalization()(conv5_2)
    conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
    conv5_3 = BatchNormalization()(conv5_3)
    conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
    conv5_3 = BatchNormalization()(conv5_3)
    pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
    drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)#Flatten and output
    flatten = Flatten(name = 'flatten')(drop5_1)
    ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)# create model 
    model = Model(inputs =visible, outputs = ouput)
    # summary layers
    print(model.summary())
    
    return model

In [20]:
model = FER_Model()
optimizer_legacy = tf.keras.optimizers.legacy.Adam(learning_rate=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer_legacy, metrics=['accuracy'])



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 48, 48, 1)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 64)        256       
 Normalization)                                                  
                                                                 
 conv1_2 (Conv2D)            (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_1 (Bat  (None, 48, 48, 64)        256       
 chNormalization)                                                
                                                                 
 pool1_1 (MaxPooling2D)      (None, 24, 24, 64)        0   

In [21]:
num_epochs = 5
history = model.fit_generator(train_flow, 
                    steps_per_epoch=len(X_train) / batch_size, 
                    epochs=num_epochs,  
                    verbose=1,  
                    validation_data=test_flow, validation_steps=len(X_test) / batch_size)

C:\Users\Acer\AppData\Local\Temp\ipykernel_65816\979276106.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_flow,


Epoch 1/5


448/448 [==============================] - 762s 2s/step - loss: 2.1674 - accuracy: 0.2283 - val_loss: 2.3896 - val_accuracy: 0.1861
Epoch 2/5
448/448 [==============================] - 781s 2s/step - loss: 1.7628 - accuracy: 0.2703 - val_loss: 1.7213 - val_accuracy: 0.3001
Epoch 3/5
448/448 [==============================] - 9256s 21s/step - loss: 1.6535 - accuracy: 0.3405 - val_loss: 1.7612 - val_accuracy: 0.3224
Epoch 4/5
448/448 [==============================] - 566s 1s/step - loss: 1.4401 - accuracy: 0.4427 - val_loss: 1.3986 - val_accuracy: 0.4466
Epoch 5/5
448/448 [==============================] - 540s 1s/step - loss: 1.3177 - accuracy: 0.4971 - val_loss: 1.7666 - val_accuracy: 0.3553


In [22]:
num_epochs = 15
history = model.fit_generator(train_flow, 
                    steps_per_epoch=len(X_train) / batch_size, 
                    epochs=num_epochs,  
                    verbose=1,  
                    validation_data=test_flow, validation_steps=len(X_test) / batch_size)

C:\Users\Acer\AppData\Local\Temp\ipykernel_65816\516418746.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_flow,


Epoch 1/15
448/448 [==============================] - 697s 2s/step - loss: 1.2471 - accuracy: 0.5280 - val_loss: 1.3400 - val_accuracy: 0.4815
Epoch 2/15
448/448 [==============================] - 846s 2s/step - loss: 1.2035 - accuracy: 0.5448 - val_loss: 1.2377 - val_accuracy: 0.5355
Epoch 3/15
448/448 [==============================] - 900s 2s/step - loss: 1.1746 - accuracy: 0.5571 - val_loss: 1.1803 - val_accuracy: 0.5561
Epoch 4/15
448/448 [==============================] - 902s 2s/step - loss: 1.1385 - accuracy: 0.5733 - val_loss: 1.1874 - val_accuracy: 0.5684
Epoch 5/15
448/448 [==============================] - 699s 2s/step - loss: 1.1224 - accuracy: 0.5781 - val_loss: 1.1386 - val_accuracy: 0.5731
Epoch 6/15
448/448 [==============================] - 525s 1s/step - loss: 1.0974 - accuracy: 0.5887 - val_loss: 1.1607 - val_accuracy: 0.5687
Epoch 7/15
448/448 [==============================] - 559s 1s/step - loss: 1.0843 - accuracy: 0.5927 - val_loss: 1.1570 - val_accuracy: 0.5773

In [23]:
num_epochs = 10
history = model.fit_generator(train_flow, 
                    steps_per_epoch=len(X_train) / batch_size, 
                    epochs=num_epochs,  
                    verbose=1,  
                    validation_data=test_flow, validation_steps=len(X_test) / batch_size)

C:\Users\Acer\AppData\Local\Temp\ipykernel_65816\1440028572.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_flow,


Epoch 1/10
448/448 [==============================] - 1024s 2s/step - loss: 0.9355 - accuracy: 0.6548 - val_loss: 1.0198 - val_accuracy: 0.6230
Epoch 2/10
448/448 [==============================] - 1017s 2s/step - loss: 0.9260 - accuracy: 0.6586 - val_loss: 0.9894 - val_accuracy: 0.6347
Epoch 3/10
448/448 [==============================] - 1015s 2s/step - loss: 0.9035 - accuracy: 0.6654 - val_loss: 1.0016 - val_accuracy: 0.6330
Epoch 4/10
448/448 [==============================] - 1035s 2s/step - loss: 0.8891 - accuracy: 0.6704 - val_loss: 1.0323 - val_accuracy: 0.6172
Epoch 5/10
448/448 [==============================] - 1025s 2s/step - loss: 0.8804 - accuracy: 0.6750 - val_loss: 0.9606 - val_accuracy: 0.6506
Epoch 6/10
448/448 [==============================] - 1023s 2s/step - loss: 0.8698 - accuracy: 0.6793 - val_loss: 1.0360 - val_accuracy: 0.6297
Epoch 7/10
448/448 [==============================] - 1026s 2s/step - loss: 0.8557 - accuracy: 0.6840 - val_loss: 1.0249 - val_accuracy:

AbortedError: Graph execution error:

Detected at node gradient_tape/model/conv5_2/Conv2D/Conv2DBackpropFilter defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\ProgramData\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\ProgramData\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\ProgramData\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\ProgramData\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line 516, 

In [24]:
num_epochs = 5
history = model.fit_generator(train_flow, 
                    steps_per_epoch=len(X_train) / batch_size, 
                    epochs=num_epochs,  
                    verbose=1,  
                    validation_data=test_flow, validation_steps=len(X_test) / batch_size)

C:\Users\Acer\AppData\Local\Temp\ipykernel_65816\979276106.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_flow,


Epoch 1/5
448/448 [==============================] - 1013s 2s/step - loss: 0.8098 - accuracy: 0.7004 - val_loss: 1.0279 - val_accuracy: 0.6322
Epoch 2/5
448/448 [==============================] - 1013s 2s/step - loss: 0.7983 - accuracy: 0.7063 - val_loss: 0.9687 - val_accuracy: 0.6590
Epoch 3/5
448/448 [==============================] - 1023s 2s/step - loss: 0.7814 - accuracy: 0.7189 - val_loss: 1.0066 - val_accuracy: 0.6428
Epoch 4/5
448/448 [==============================] - 1035s 2s/step - loss: 0.7665 - accuracy: 0.7198 - val_loss: 0.9992 - val_accuracy: 0.6481
Epoch 5/5
448/448 [==============================] - 1051s 2s/step - loss: 0.7599 - accuracy: 0.7232 - val_loss: 1.0221 - val_accuracy: 0.6278


In [25]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [1]:
from tensorflow.keras.models import model_from_json
model = model_from_json(open("model.json", "r").read())
model.load_weights('model.h5')

In [2]:
import cv2
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [41]:
# import cv2
# import numpy as np
# from keras.preprocessing.image import img_to_array
# cap=cv2.VideoCapture(0)
# while cap.isOpened():
#     res,frame=cap.read()
#     height, width , channel = frame.shape
#     gray_image= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_haar_cascade.detectMultiScale(gray_image)
#     try:
#         for (x,y, w, h) in faces:
#             cv2.rectangle(frame, pt1 = (x,y),pt2 = (x+w, y+h), color = (255,0,0),thickness =  2)
#             roi_gray = gray_image[y-5:y+h+5,x-5:x+w+5]
#             roi_gray=cv2.resize(roi_gray,(48,48))
#             image_pixels = img_to_array(roi_gray)
#             image_pixels = np.expand_dims(image_pixels, axis = 0)
#             image_pixels /= 255
            
#             predictions = model.predict(image_pixels)
#             max_index = np.argmax(predictions[0])
#             emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
#             emotion_prediction = emotion_detection[max_index]
            
            
#     except Exception as e:
#         print(f"Error: {e}")

#     cv2.imshow('Emotion Detection', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()        

In [42]:
# import cv2
# import numpy as np
# from keras.preprocessing.image import img_to_array

# # Assuming 'face_haar_cascade', 'model', and other necessary imports are defined earlier

# cap = cv2.VideoCapture(0)

# while cap.isOpened():
#     res, frame = cap.read()
#     height, width, channel = frame.shape

#     # Creating an Overlay window to write prediction and confidence
#     sub_img = frame[0:int(height/6), 0:int(width)]
#     black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0
#     res = cv2.addWeighted(sub_img, 0.77, black_rect, 0.23, 0)

#     FONT = cv2.FONT_HERSHEY_SIMPLEX
#     FONT_SCALE = 0.8
#     FONT_THICKNESS = 2
#     lable_color = (10, 10, 255)
#     lable = "Emotions"
#     lable_dimension = cv2.getTextSize(lable, FONT, FONT_SCALE, FONT_THICKNESS)[0]
#     textX = int((res.shape[1] - lable_dimension[0]) / 2)
#     textY = int((res.shape[0] + lable_dimension[1]) / 2)
#     cv2.putText(res, lable, (textX, textY), FONT, FONT_SCALE, (0, 0, 0), FONT_THICKNESS)

#     # Prediction part
#     gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_haar_cascade.detectMultiScale(gray_image)
#     try:
#         for (x, y, w, h) in faces:
#             cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 0), thickness=2)
#             roi_gray = gray_image[y-5:y+h+5, x-5:x+w+5]
#             roi_gray = cv2.resize(roi_gray, (48, 48))
#             image_pixels = img_to_array(roi_gray)
#             image_pixels = np.expand_dims(image_pixels, axis=0)
#             image_pixels /= 255
#             predictions = model.predict(image_pixels)
#             max_index = np.argmax(predictions[0])
#             emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
#             emotion_prediction = emotion_detection[max_index]
#             cv2.putText(res, "Sentiment: {}".format(emotion_prediction), (0, textY+22+5), FONT, 0.7, lable_color, 2)
#             #lable_violation = 'Confidence: {}'.format(str(np.round(np.max(predictions[0])*100, 1)) + "%")
#             violation_text_dimension = cv2.getTextSize(lable_violation, FONT, FONT_SCALE, FONT_THICKNESS)[0]
#             violation_x_axis = int(res.shape[1] - violation_text_dimension[0])
#             cv2.putText(res, lable_violation, (violation_x_axis, textY+22+5), FONT, 0.7, lable_color, 2)
#     except:
#         pass

#     frame[0:int(height/6), 0:int(width)] = res
#     cv2.imshow('frame', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array

# Assuming 'face_haar_cascade', 'model', and other necessary imports are defined earlier

cap = cv2.VideoCapture(0)

while cap.isOpened():
    res, frame = cap.read()
    height, width, channel = frame.shape

    # Create a transparent overlay with a completely transparent green rectangle
    overlay = np.zeros_like(frame, dtype=np.uint8)
    rect_start = (0, 0)
    rect_end = (width, int(height/6))
    cv2.rectangle(overlay, rect_start, rect_end, (0, 255, 0, 0), cv2.FILLED)

    # Prediction part
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray_image)
    
    try:
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
            roi_gray = gray_image[y-5:y+h+5, x-5:x+w+5]
            roi_gray = cv2.resize(roi_gray, (48, 48))
            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)
            image_pixels /= 255
            predictions = model.predict(image_pixels)
            max_index = np.argmax(predictions[0])
            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]

            # Place the detected emotion with the moving face
            text_size = cv2.getTextSize(emotion_prediction, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
            textX_emotion = int(x + (w - text_size[0]) / 2)
            textY_emotion = int(y - 10)

            cv2.putText(frame, f"Detected Emotion: {emotion_prediction}", (textX_emotion, textY_emotion), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    except:
        pass

    # Combine the frame and overlay
    result = cv2.addWeighted(frame, 1, overlay, 0, 0)

    cv2.imshow('frame', result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 31ms/step


In [3]:
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array

# Assuming 'face_haar_cascade', 'model', and other necessary imports are defined earlier

cap = cv2.VideoCapture(0)
emotion_results = []

# Set the capture duration to 10 seconds
capture_duration = 10  # in seconds
start_time = cv2.getTickCount()

while cap.isOpened():
    # Calculate elapsed time
    elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()

    if elapsed_time >= capture_duration:
        break

    res, frame = cap.read()
    height, width, channel = frame.shape

    # Create a transparent overlay with a completely transparent green rectangle
    overlay = np.zeros_like(frame, dtype=np.uint8)
    rect_start = (0, 0)
    rect_end = (width, int(height/6))
    cv2.rectangle(overlay, rect_start, rect_end, (0, 255, 0, 0), cv2.FILLED)

    # Prediction part
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray_image)
    
    try:
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(0, 255, 0), thickness=2)
            roi_gray = gray_image[y-5:y+h+5, x-5:x+w+5]
            roi_gray = cv2.resize(roi_gray, (48, 48))
            image_pixels = img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis=0)
            image_pixels /= 255
            predictions = model.predict(image_pixels)
            max_index = np.argmax(predictions[0])
            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]

            # Store detected emotions in a list
            emotion_results.append(emotion_prediction)

            # Place the detected emotion with the moving face
            text_size = cv2.getTextSize(emotion_prediction, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
            textX_emotion = int(x + (w - text_size[0]) / 2)
            textY_emotion = int(y - 10)

            cv2.putText(frame, f"Detected Emotion: {emotion_prediction}", (textX_emotion, textY_emotion), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    except:
        pass

    # Combine the frame and overlay
    result = cv2.addWeighted(frame, 1, overlay, 0, 0)

    cv2.imshow('frame', result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Determine the final output based on the majority emotion in the list
final_emotion = max(set(emotion_results), key=emotion_results.count)
print(f"Final Emotion: {final_emotion}")

1/1 [==============================] - 0s 429ms/step
Final Emotion: sad
